In [11]:
from open_flamingo import create_model_and_transforms
from huggingface_hub import hf_hub_download
import torch
from PIL import Image
import requests
import torch
import json
import random

In [12]:
def model_setup():
    model, image_processor, tokenizer = create_model_and_transforms(
        clip_vision_encoder_path="ViT-L-14",
        clip_vision_encoder_pretrained="openai",
        lang_encoder_path="anas-awadalla/mpt-1b-redpajama-200b",
        tokenizer_path="anas-awadalla/mpt-1b-redpajama-200b",
        cross_attn_every_n_layers=1
    )

    checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-3B-vitl-mpt1b", "checkpoint.pt")
    model.load_state_dict(torch.load(checkpoint_path), strict=False)
    
    return model, image_processor, tokenizer

In [13]:
def getImage(num, annotated):
    if annotated:
        return Image.open("../ai2d/annotated_images/" + str(num) + ".png")
    return Image.open("../ai2d/images/" + str(num) + ".png")

In [14]:
def getQuestion(num):
    f = open("../ai2d/questions/" + str(num) + ".png.json")
 
    # returns JSON object as a dictionary
    data = json.load(f)
    
    dictionary = data["questions"]
    question = list(dictionary.keys())[0]

    correctAnswerIndex = dictionary[question]["correctAnswer"]
    correctAnswer = dictionary[question]["answerTexts"][correctAnswerIndex]

    # Closing file
    f.close()

    return question, correctAnswer

In [15]:
def preprocessImage(image_processor, image):
    vision_x = [image_processor(image).unsqueeze(0)]
    vision_x = torch.cat(vision_x, dim=0)
    vision_x = vision_x.unsqueeze(1).unsqueeze(0)
    return vision_x

In [16]:
def preprocessText(tokenizer, question):
    lang_x = tokenizer(
        ["<image>" + question + " The answer is"],
        return_tensors="pt",
    )
    return lang_x

In [ ]:
from store_annotations import annotate

for i in range(2557, 4097):
    print(i)
    ann_path = "../ai2d/annotations/"+str(i)+".png.json"
    img_path = "../ai2d/images/"+str(i)+".png"
    out_path = "../ai2d/annotated_images/"+str(i)+".png"
    annotate(ann_path, img_path, out_path)

In [17]:
def generate_text(model, vision_x, lang_x, tokenizer):
    generated_text = model.generate(
        vision_x=vision_x,
        lang_x=lang_x["input_ids"],
        attention_mask=lang_x["attention_mask"],
        max_new_tokens=20,
        num_beams=3,
    )
    return tokenizer.decode(generated_text[0])

In [18]:
model, image_processor, tokenizer = model_setup()
tokenizer.padding_side = "left"

Using pad_token, but it is not set yet.


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50280. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Flamingo model initialized with 1046992944 trainable parameters


In [29]:
for i in range(100):
    num = random.randint(0, 2555)
    try:
        print("Image", str(num))
        image = getImage(num, False)
        annotated_image = getImage(num, True)
        question, correctAnswer = getQuestion(num)
        print(question)
        lang_x = preprocessText(tokenizer, question)

        print("REGULAR")
        vision_x = preprocessImage(image_processor, image)
        reg_answer = generate_text(model, vision_x, lang_x, tokenizer)
        print(reg_answer)

        print("ANNOTATED")
        vision_x = preprocessImage(image_processor, annotated_image)
        annotated_answer = generate_text(model, vision_x, lang_x, tokenizer)
        print(annotated_answer)

        print("CORRECT", correctAnswer)

        file_path = "../ai2d/responses/" + str(num) + ".txt"
        with open(file_path, "w") as file:
            # Write the text to the file
            text_to_save = reg_answer + "\n" + annotated_answer
            file.write(text_to_save)

        # start = len("<image>") + len(question) + len("The answer is")
        # end = -1 * len("<|endofchunk|>")
        # print(similarityScore(eval_model, correctAnswer, answer[start, end]))
    except:
        continue

Image 2073
Based on the above food web, what will happen if the number of algae decreases.
REGULAR


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


Image 1129
Which forms the calyx of a flower?
REGULAR


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


<image>Which forms the calyx of a flower? The answer is the pistil.<|endofchunk|>
ANNOTATED


...

Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


<image>What organism is showed in the diagram above? The answer is:<|endofchunk|>
CORRECT Plant
Image 2205
How many stage shown?
REGULAR


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


Image 14
According to the given food chain, if sea weeds are washed out due to Tsunami, which one would happen?
REGULAR


Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


In [ ]:
# pip install -U sentence-transformers

# from sentence_transformers import SentenceTransformer, util

# eval_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# def similarityScore(eval_model, realAnswer, modelOutput):
#     embedding_1= eval_model.encode(realAnswer, convert_to_tensor=True)
#     embedding_2 = eval_model.encode(modelOutput, convert_to_tensor=True)
#     return util.pytorch_cos_sim(embedding_1, embedding_2)